# Init

## Imports

In [ ]:
# Python modules
import itertools
import time

# Data management
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pylab as plt
import seaborn as sns
sns.set(style="ticks", font="Source Sans Pro", font_scale=.9, palette="muted")

# Fitting
import scipy.optimize as opt

# Project modules
import luescher_nd.utilities as ut
import luescher_nd.zeta as zeta

In [ ]:
%load_ext blackcellmagic

## Parameters

Quantities will be expressed in units of [fm]. Energies are converted to inverse fermi using $\hbar c = 197.326$ MeV/fm.

In [ ]:
HBARC = 197.326 # MeV / fm
M_NUCLEON = (938.27 + 939.57) / 2 / HBARC # in fm^{-1}

In [ ]:
def energy2x(energy, mass, L):
    """
    """
    x_ = np.sqrt(np.abs(energy) * 2 * mass) * L / 2 / np.pi
    x_ *= x_
    
    return np.where(energy > 0, x_, -x_)

## 2-D

## Functions

Test the convergence of the 2-D zeta function

In [ ]:
x = np.linspace(-4, 10, 1000)

lambdas = [100, 200, 500]

data = []

for l in lambdas:
    for xi, yi in zip(x, zeta.zeta2(x, l)):
        assert zeta.zeta2([xi], l) == yi
        data.append({"x": xi, "y": yi, "l": l})

z2_data = pd.DataFrame(data)

Extract numerical divergencies of $S_2$

In [ ]:
divergencies = np.unique( np.sum(
        np.array(
            list(itertools.product(*[np.arange(int(x.min()), int(x.max()))] * 2)),
            dtype=float,
        )
        ** 2,
        axis=1,
    ) 
)

Find zeros of $S_2$

In [ ]:
lmax = z2_data.l.max()

n_zeros = 8
lower = -2
zeros = []
for upper in divergencies[:n_zeros]:
    eps = (upper - lower) / 20 
    res = opt.minimize_scalar(
        lambda x: zeta.zeta2([x], 2000)**2, 
        bounds=(lower + eps, upper - eps),
        method="Bounded"
    )
    zeros.append(res.x)
    lower = upper

In [ ]:
def get_laplace_coefficients(n_step: int):
    """Implementation fo finite step laplace derivative coefficients.

    See also https://en.wikipedia.org/wiki/Finite_difference_coefficient.

    Arguments
    ---------
        n_step: int
            Step range of derivative. Must be larger than 0.
    """
    out = {}

    if n_step < 1:
        raise ValueError("'n_step' must be larger than zero.")

    elif n_step == 1:
        out[0] = -2
        out[1] = out[-1] = 1

    elif n_step == 2:
        out[0] = -5 / 2
        out[1] = out[-1] = 4 / 3
        out[2] = out[-2] = -1 / 12

    elif n_step == 3:
        out[0] = -49 / 18
        out[1] = out[-1] = 3 / 2
        out[2] = out[-2] = -3 / 20
        out[3] = out[-3] = 1 / 90

    elif n_step == 4:
        out[0] = -205 / 72
        out[1] = out[-1] = 8 / 5
        out[2] = out[-2] = -1 / 5
        out[3] = out[-3] = 8 / 315
        out[4] = out[-4] = -1 / 560

    else:
        raise NotImplementedError("'n_step' not implemented for values larger than 6.")

    return out

In [ ]:
fig_zeta_2d, axs = plt.subplots(dpi=300, figsize=(4,3), ncols=1, sharey=True, sharex=True)

for ax in [axs]:
    ax.axhline(0, ls="-", c="grey", lw=.5, alpha=.5)

    for div in divergencies:
        if div < x.max():
            ax.axvline(div, ls="--", c="grey", lw=.5, alpha=.5)

    grouped = z2_data.groupby("l")
    for key, group in grouped:
        group.plot(ax=ax, x='x', y='y', label=key, lw=1, ls="None", marker=".", ms=.5)

    for zero in zeros:
        if zero < x.max():
            ax.scatter(zero, 0, marker="x", c="black", lw=.5, s=5, zorder=10)

    ax.set_ylim(-10, 10)
    ax.legend(loc="best", fontsize="xx-small", title="$\Lambda$")

    ax.set_xlabel("$x$")
    ax.set_ylabel("$S_2(x)$")

plt.tight_layout()
plt.show(fig_zeta_2d)


## Fitting

Fit $c_0(a)$ such that $E = E(x_0, L)$, where

$$x_0 = \left( \frac{\gamma L}{2 \pi} \right)^2 = \frac{\mu E L^2}{2\pi^2} = -1.25107628$$

In [ ]:
mu = M_NUCLEON / 2
L = 1.
E0 = zeros[0] * 2 * np.pi**2 / mu / L**2

Fit

In [ ]:
epsilons = [0.1, 0.08, 0.05, 0.02, 0.01]

data = []

for n_step in range(1, 4):
    for epsilon in epsilons:
        n1d_max = int(L / epsilon)
        solver = ut.Solver(
            n1d_max=n1d_max,
            ndim_max=2,
            lattice_spacing=epsilon,
            derivative_shifts=get_laplace_coefficients(n_step),
        )

        func = lambda c0: (solver.get_energies(c0, n_energies=1) - E0) ** 2

        c0 = -1.

        res = opt.minimize(func, c0)
        energies = solver.get_energies(res.x, n_energies=24)

        data.append({"a": epsilon, "c": res.x[0], "e": energies, "n": n_step})

df = pd.DataFrame(data)

Plot

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(4, 3))

ax.axhline(0, ls="-", c="grey", lw=.5, alpha=.5)

for div in divergencies:
    if div < x.max():
        ax.axvline(div, ls="--", c="grey", lw=.5, alpha=.5)

group = z2_data.query("l == @lmax")
group.plot(ax=ax, x="x", y="y", ls="None", ms=.5, marker=".", label="")

y_min = -2
y_max = +2

nc = 0

markers = ["o", "x", "s", "d", "+"]

for epsilon, energies, n_step in df[["a", "e", "n"]].get_values():
    x0 = energy2x(energies, mu, L)
    y0 = zeta.zeta2(x0, 2000)

    
    n1d_max = int( L / epsilon)
    ax.plot(
        x0 + (2 - n_step) / 4,
        y0,
        markers[n_step - 1],
        ms=3,
        lw=.5,
        label=rf"$\frac{{L}}{{\epsilon}}={n1d_max}$ @ $n_S = {n_step}$",
    )

    if nc == len(epsilons) - 1:
        for i, (xi, yi) in enumerate(zip(x0, y0)):
            if yi < y_max and yi > y_min and xi < x.max() and xi > x.min():
                ax.text(
                    xi,
                    yi,
                    i,
                    fontdict={"size": 5},
                    ha="right",
                    va="bottom",
                    withdash=True,
                )
    nc += 1

ax.set_ylim(y_min, y_max)
ax.legend(loc="upper right", fontsize=5, bbox_to_anchor=(1.3, 1.0))

ax.set_xlabel("$x$")
ax.set_ylabel("$S_2(x)$")

plt.show(fig)
